![hslu_logo.png](img/hslu_logo.png)

## Week 01

<hr style="border:1px solid black">

# Exercise: Introduction to PyTorch
---
---

*PyTorch is a machine learning framework based on the Torch library,used for applications such as computer vision and natural language processing,originally developed by Meta AI and now part of the Linux Foundation umbrella. It is recognized as one of the two most popular machine learning libraries alongside TensorFlow, offering free and open-source software released under the modified BSD license. Although the Python interface is more polished and the primary focus of development, PyTorch also has a C++ interface.* [Wikipedia](https://en.wikipedia.org/wiki/PyTorch).

Due to its similarity with numpy - with regard to array (or tensor) management - you can quickly familiarize with PyTorch if you have a good knowledge of numpy. 

### Import PyTorch and numpy

In [75]:
import torch
import numpy as np

### Handling of tensors
#### Convert from numpy

In [76]:
array_np = np.ones([3,2])
print(array_np)

array_py = torch.from_numpy(array_np)
print(array_py)

[[1. 1.]
 [1. 1.]
 [1. 1.]]
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]], dtype=torch.float64)


#### Create from scratch

In [77]:
x = torch.ones([3,2],dtype=torch.float)
print(x)

y = torch.rand([2,3],dtype=torch.float)
print(y)


tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[0.4880, 0.3375, 0.1098],
        [0.4233, 0.0219, 0.6287]])


#### Operation on tensors 

Observe the following multiplications. The multiplication sign `*` represents an element-wise multiplication (including possible broadcasting as in numpy) and `@` is a multiplication in the mathematical sense i.e., matrix multiplication.


In [78]:
print(x*y.T)  # Transponierte --> Komponentenweise Multiplikation
print(x@y)
print(y@x)
print(y.T@x.T)


tensor([[0.4880, 0.4233],
        [0.3375, 0.0219],
        [0.1098, 0.6287]])
tensor([[0.9113, 0.3594, 0.7385],
        [0.9113, 0.3594, 0.7385],
        [0.9113, 0.3594, 0.7385]])
tensor([[0.9353, 0.9353],
        [1.0739, 1.0739]])
tensor([[0.9113, 0.9113, 0.9113],
        [0.3594, 0.3594, 0.3594],
        [0.7385, 0.7385, 0.7385]])


#### Indexing

Indexing of torch tensors is as in numpy

In [79]:
print(y[:,0])
print(y[1,:])
print(y > 0.5)

tensor([0.4880, 0.4233])
tensor([0.4233, 0.0219, 0.6287])
tensor([[False, False, False],
        [False, False,  True]])


#### Assignemnt / Copy

Care has to be taken - as in numpy - when using an assignment of a `x` to a tensor `y`. This does $not$ represent a deep copy and changes to the elements of `y` will also change `x`.

In [80]:
x = torch.ones([3,2],dtype=torch.float)
y = torch.rand([2,3],dtype=torch.float)
y = x # call by reference
print(x)
print(y)
y[0,0] = 2 # ändert auch x
print(x)
print(y)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[2., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[2., 1.],
        [1., 1.],
        [1., 1.]])


Use a deep copy with the `copy_` operator to ensure a new tensor with own elements.

In [81]:
x = torch.ones([3,2],dtype=torch.float)
y = torch.rand([3,2],dtype=torch.float)
y.copy_(x)
print(x)
print(y)
y[0,0] = 2
print(x)
print(y)

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([[2., 1.],
        [1., 1.],
        [1., 1.]])


### Autograd functionality
PyTorch provides the possibility to automatically keep track of the gradients with respect to certain tensors. In the following example, PyTorch will keep track of the gradients with respect to `W` and `B`when calculating the result `res`. This is done by calling `res.backward()`. The gradients are then available in the `grad` member of the respective tensor.

#### Automatic gradient using autograd

In [94]:
x = torch.tensor([[1,2,3]],dtype=torch.double).T
#declear W and b as tensors with gradient determination
W = torch.randn([2,3],dtype=torch.double, requires_grad=True) # requires_grad=True --> we want to calculate the gradient of W
b = torch.randn([2,1],dtype=torch.double, requires_grad=True) 

print(x)
print(W)
print(b)

#calculate a function called 'cost'
a = W@x + b         # matrix multiplication 
print('a=',a, a.shape)
cost = a.T@a        # quadratstumme 
print('cost=',cost.item())

#now call backward() on 'cost' to determine the gradients of W and b
cost.backward()     # berechnet für alle verwendeten variablen die partiellen ableitungen, diese stehen dann in W.grad und b.grad zur verfügung

#print the result
print(W.grad.numpy())
print(b.grad.numpy())

tensor([[1.],
        [2.],
        [3.]], dtype=torch.float64)
tensor([[ 0.4002,  0.5487,  0.9007],
        [-0.1654,  0.1927, -1.3228]], dtype=torch.float64, requires_grad=True)
tensor([[-0.5270],
        [-0.2213]], dtype=torch.float64, requires_grad=True)
a= tensor([[ 3.6726],
        [-3.9696]], dtype=torch.float64, grad_fn=<AddBackward0>) torch.Size([2, 1])
cost= 29.245965535544048
[[  7.34516629  14.69033258  22.03549887]
 [ -7.93929432 -15.87858864 -23.81788297]]
[[ 7.34516629]
 [-7.93929432]]


It is important to notice, that the `grad` member is not cleared automatically before a call to `backward()`. Thus successive calls will accumulate the result

In [96]:
#repeat the same calculation 
a = W@x + b
cost = a.T@a
print(cost.item())

#now call backward() on 'cost' to determine the gradients of W and b
cost.backward()

#print the result
print(W.grad.numpy())
print(b.grad.numpy())

#it will differ and increase with each call --> Das Resultat wird bei jedem Aufruf von backward() addiert --> Deswegen sollte W.grad = None vor jedem Aufruf gesetzt werden

29.245965535544048
[[ 22.03549887  44.07099773  66.1064966 ]
 [-23.81788297 -47.63576593 -71.4536489 ]]
[[ 22.03549887]
 [-23.81788297]]


Thus, the `grad` member of `W` and `b` must be reset before each call to `backward()`:

In [97]:
#repeat the same calculation 
a = W@x + b
cost = a.T@a
print(cost.item())

#clear the grad entry of W and b
W.grad = None
b.grad = None

#now call backward() on 'cost' to determine the gradients of W and b
cost.backward()

#print the result
print(W.grad.numpy())
print(b.grad.numpy())

#now the results will be again identical as in the first call

29.245965535544048
[[  7.34516629  14.69033258  22.03549887]
 [ -7.93929432 -15.87858864 -23.81788297]]
[[ 7.34516629]
 [-7.93929432]]


A further problem may arise, once you want to use the values of `W` and `b` for calculation. Variables that required gradient calculations are restricted in their use. But you can always call `with torch.no_grad()` in order to suppress the gradient calculation.

In [85]:
#the following call (W += ...) will raise an error as long as the first statement (with torch.no_grad():) is commented
#
with torch.no_grad():
    W += W.grad
    W -= W.grad

#### Manual verification of gradient determination

For comparison we determine the gradient manually. Recall that 
$$
res = \mathbf{a}^T \cdot \mathbf{a} = (\mathbf{W} \cdot \mathbf{x} + \mathbf{b})^T \cdot (\mathbf{W} \cdot \mathbf{x} + \mathbf{b}) 
$$
Thus it is straight forward to show that:
$$
\frac{\partial}{\partial \mathbf{W}} res = 2 \cdot (\mathbf{W} \cdot \mathbf{x} + \mathbf{b}) \cdot \mathbf{x}^T = 2\cdot \mathbf{a} \cdot \mathbf{x}^T
$$
$$
\frac{\partial}{\partial \mathbf{b}} res = 2 \cdot (\mathbf{W} \cdot \mathbf{x} + \mathbf{b})= 2\cdot \mathbf{a}
$$

In [86]:
with torch.no_grad():
    W_grad = 2*a*x.T # Transponierte --> Komponentenweise Multiplikation
    b_grad = 2*a
    print(W_grad.numpy())
    print(b_grad.numpy())

[[-2.41061076 -4.82122152 -7.23183229]
 [-1.4430012  -2.8860024  -4.3290036 ]]
[[-2.41061076]
 [-1.4430012 ]]


#### Numeric gradient determination

Finally, again for comparison we determine the gradient numerically.

In [87]:
eps = 1e-7
W_grad = torch.zeros(W.shape,dtype=torch.double)
for row in range(0, W.shape[0]):
    for col in range(0, W.shape[1]):
        dw = torch.zeros(W.shape,dtype=torch.double)
        dw[row,col] = eps
        a_eps = (W+dw)@x + b
        cost_eps = a_eps.T@a_eps
        W_grad[row,col] = (cost_eps - cost)/eps
        
print(W_grad)

b_grad = torch.zeros(b.shape,dtype=torch.double)
for row in range(0, b.shape[0]):
    db = torch.zeros(b.shape,dtype=torch.double)
    db[row,0] = eps
    a_eps = W@x + b + db
    cost_eps = a_eps.T@a_eps
    b_grad[row,0] = (cost_eps - cost)/eps

print(b_grad)

tensor([[-2.4106, -4.8212, -7.2318],
        [-1.4430, -2.8860, -4.3290]], dtype=torch.float64,
       grad_fn=<CopySlices>)
tensor([[-2.4106],
        [-1.4430]], dtype=torch.float64, grad_fn=<CopySlices>)
